## Simulating encounters

If this is your first time using a notebook, please ensure you have [Python installed](https://www.python.org/downloads/) and you have installed the additional dependencies with the following command: `pip install -r ./automation/requirements.txt`

First, we'll navigate to the repo's root:


In [1]:
import os

if os.path.basename(os.getcwd()) != "TheGame":
    os.chdir("..")

Next, import what you need:


In [2]:
import copy
from automation.templates.bestiary import Bestiary
from automation.simulator.deck import Deck
from automation.simulator.encounter import Encounter

Use the Bestiary to provide dictionaries for each of the combatants. We can use emojis to represent them in the combat output. The Name value will be shown in the log.

Note that files with SAMPLE in the name are designed to workshop ideas and test edge cases, not be included in the game.


In [3]:
b = Bestiary(input_files="06_Bestiary_SAMPLE.yaml").raw_data
c1 = b["Clubs1"]
c1.update(dict(Name="💀", id="A"))
c2 = copy.copy(b["Clubs1"])
c2.update(dict(Name="👽", id="B"))
s = b["Spider Queen"]
s.update(dict(Name="🎇", id="C"))

Initialize the encounter. We can see who is involved and resource information by looking at the encounter's `PCs`, `enemies`, or `turn_order` properties.

Note that this may differ from a real encounter because each participant has their own hand and deck. In a true encounter, a GM might manage many characters with the same deck.


In [4]:
e = Encounter(PCs=[c1, c2], Enemies=[s])
e.turn_order

[👽
 TC       : ♥️ 5 | pc.HP : 7/7
 Hand     :  02 | pc.PP : 2/2
 Deck     :  51 | pc.AP : 2/2
 Discards :  01 | RestC : 7/7,
 💀
 TC       : ♦️ 4 | pc.HP : 7/7
 Hand     :  02 | pc.PP : 2/2
 Deck     :  51 | pc.AP : 2/2
 Discards :  01 | RestC : 7/7,
 🎇
 TC       : ♥️ T | pc.HP : 8/8
 Hand     :  02 | pc.PP : 6/6
 Deck     :  51 | pc.AP : 2/2
 Discards :  01 | RestC : 8/8]

Next, we can simulate a couple rounds of combat. Here, each participant will choose a Power available to them (if sufficient PP) and apply it to an enemy at random. This does not yet cover buffing powers (e.g., Shield, Lend Aid) or mind control status effects (i.e., Charmed, Enthralled).


In [5]:
e.sim_round(3)

[16:38][INFO]: 👽 used 0/2 PP with Attack, Dual-Wield
[16:38][INFO]: 👽 used 0/2 PP with Attack, Weapon
[16:38][INFO]: 💀 used 0/2 PP with Attack, Dual-Wield
[16:38][INFO]: 💀 used 0/2 PP with Attack, Weapon
[16:38][INFO]: 💀 wounded 🎇 by 1: AP 1/2, HP 8/8
[16:38][INFO]: 🎇 used 0/6 PP with Sear
[16:38][INFO]: 👽 is Burned: 1
[16:38][INFO]: 🎇 used 0/6 PP with Multi-Attack 2
[16:38][INFO]: 🎇 wounded 💀 by 1: AP 1/2, HP 7/7
[16:38][INFO]: 🎇 wounded 👽 by 1: AP 1/2, HP 7/7
[16:38][INFO]: 👽 used 1/2 PP with Attack, Charge
[16:38][INFO]: 👽 wounded 🎇 by 2: AP 0/2, HP 7/8
[16:38][INFO]: 👽 shakes off Burned
[16:38][INFO]: 💀 used 1/2 PP with Attack, Charge
[16:38][INFO]: 💀 wounded 🎇 by 2: AP 0/2, HP 5/8
[16:38][INFO]: 💀 used 0/1 PP with Attack, Dual-Wield
[16:38][INFO]: 🎇 used 0/6 PP with Sear
[16:38][INFO]: 💀 is Burned: 1
[16:38][INFO]: 🎇 used 0/6 PP with Multi-Attack 2
[16:38][INFO]: 🎇 wounded 👽 by 1: AP 0/2, HP 7/7
[16:38][INFO]: 🎇 wounded 👽 by 1: AP 0/2, HP 6/7
[16:38][INFO]: 👽 used 0/1 PP with Atta

We can even simulate epic events.


In [6]:
e.sim_epic_event(successes_needed=3)

[16:38][INFO]: Party 0, GM 1 | GM Suited Hit
[16:38][INFO]: Party 1, GM 1 | 👽 Suited Hit
[16:38][INFO]: Party 2, GM 1 | 💀 Suited Hit
[16:38][INFO]: Party 2, GM 2 | GM Suited Hit
[16:38][INFO]: Party 3, GM 2 | 💀 Major Success
[16:38][INFO]: Party 4, GM 2 | 👽 Suited Hit
[16:38][INFO]: Party wins after 18 total cards drawn


Let's see how everyone is doing.


In [7]:
e.turn_order

[👽
 TC       : ♥️ 5 | pc.HP : 5/7
 Hand     :  03 | pc.PP : 1/2
 Deck     :  36 | pc.AP : 0/2
 Discards :  15 | RestC : 7/7,
 💀
 TC       : ♦️ 4 | pc.HP : 7/7
 Hand     :  05 | pc.PP : 0/2
 Deck     :  33 | pc.AP : 1/2
 Discards :  16 | RestC : 7/7,
 🎇
 TC       : ♥️ T | pc.HP : 2/8
 Hand     :  02 | pc.PP : 5/6
 Deck     :  44 | pc.AP : 0/2
 Discards :  08 | RestC : 8/8]

If you're interested in draining a specific character's resources, we can index them directly and subtract values.


In [8]:
spider_queen = e.enemies[0]
spider_queen.HP = 1
spider_queen

🎇
TC       : ♥️ T | pc.HP : 1/8
Hand     :  02 | pc.PP : 5/6
Deck     :  44 | pc.AP : 0/2
Discards :  08 | RestC : 8/8

The only thing that can't be set directly is the deck. This has to be managed by either drawing, using a fate card, or shuffling.


In [9]:
spider_queen.draw()

♠️ 2

In [10]:
spider_queen.exchange_fate()

[16:38][INFO]: Exchanged Fate Card: ⬛ JOKER


In [11]:
spider_queen.shuffle(2)
spider_queen

🎇
TC       : ♦️ 5 | pc.HP : 1/8
Hand     :  01 | pc.PP : 5/6
Deck     :  44 | pc.AP : 0/2
Discards :  08 | RestC : 8/8

We can give some or all participants a rest.


In [12]:
e.sim_quick_rest(participants=e.PCs)  # If no participants specified, all

[16:38][INFO]: 👽 recovered 6 HP/PP/AP during Quick Rest
[16:38][INFO]: 💀 recovered 4 HP/PP/AP during Quick Rest
[16:38][INFO]: 🎇 recovered 9 HP/PP/AP during Quick Rest


In [13]:
e.turn_order

[👽
 TC       : ♠️ 3 | pc.HP : 8/7
 Hand     :  05 | pc.PP : 2/2
 Deck     :  43 | pc.AP : 2/2
 Discards :  06 | RestC : 4/7,
 💀
 TC       : ♦️ 8 | pc.HP : 8/7
 Hand     :  05 | pc.PP : 2/2
 Deck     :  41 | pc.AP : 2/2
 Discards :  08 | RestC : 5/7,
 🎇
 TC       : ♦️ Q | pc.HP : 8/8
 Hand     :  01 | pc.PP : 6/6
 Deck     :  48 | pc.AP : 1/2
 Discards :  04 | RestC : 3/8]

Or a full rest.


In [14]:
e.sim_full_rest()

In [15]:
e.turn_order

[👽
 TC       : ♠️ 2 | pc.HP : 7/7
 Hand     :  05 | pc.PP : 2/2
 Deck     :  48 | pc.AP : 2/2
 Discards :  01 | RestC : 7/7,
 💀
 TC       : ♥️ K | pc.HP : 7/7
 Hand     :  05 | pc.PP : 2/2
 Deck     :  48 | pc.AP : 2/2
 Discards :  01 | RestC : 7/7,
 🎇
 TC       : ♠️ 8 | pc.HP : 8/8
 Hand     :  02 | pc.PP : 6/6
 Deck     :  51 | pc.AP : 2/2
 Discards :  01 | RestC : 8/8]

## Logging

Much of the information we're seeing above in the `[TIME][INFO]` format is as a result of the [logger](https://www.loggly.com/ultimate-guide/python-logging-basics/) that keeps track of this information. We can adjust what we see by adjusting the log level.


In [16]:
from automation.utils import logger

logger.setLevel("DEBUG")  # Most information
# logger.setLevel('INFO') # Standard information
# logger.setLevel('WARNING') # Only problems
# logger.setLevel('CRITICAL') # No information
spider_queen.save(DR=3, attrib=["AGL", "STR"], upper_lower="lower", draw_n=2)
e.sim_round()

[16:38][DEBUG]: Drew [♦️ J, ♠️ 2] vs ♠️ 8 with TR 4 at Lower Hand 2: Suited Miss
[16:38][INFO]: 👽 used 0/2 PP with Attack, Weapon
[16:38][DEBUG]: Drew [♠️ 8] vs ♠️ 8 with TR 3: Major Success
[16:38][INFO]: 👽 wounded 🎇 by 2: AP 0/2, HP 8/8
[16:38][INFO]: 👽 used 0/2 PP with Drown
[16:38][DEBUG]: Drew [♦️ 6] vs ♠️ 8 with TR 0: Miss
[16:38][INFO]: 🎇 is Suffocating: 1
[16:38][INFO]: 💀 used 1/2 PP with Attack, Charge
[16:38][DEBUG]: Drew [♥️ 2] vs ♠️ 8 with TR 3: Miss
[16:38][INFO]: 💀 used 0/1 PP with Attack, Dual-Wield
[16:38][DEBUG]: Drew [♠️ K, ♥️ 8, ♠️ 8] vs ♠️ 8 with TR 3 at Lower Hand 3: Suited Miss
[16:38][DEBUG]: Drew [♠️ T] vs ♠️ 8 with TR 5: Suited Hit
[16:38][INFO]: 🎇 shakes off Stunned
[16:38][INFO]: 🎇 used 0/6 PP with Sear
[16:38][DEBUG]: Drew [♠️ 3] vs ♠️ 2 with TR 0: Suited Miss
[16:38][INFO]: 👽 is Burned: 1
[16:38][INFO]: 🎇 used 0/6 PP with Multi-Attack 2
[16:38][DEBUG]: Drew [♥️ 8] vs ♥️ K with TR 4: Suited Miss
[16:38][DEBUG]: Drew [♠️ 3] vs ♥️ K with TR 4: Hit
[16:38][INFO

For a more detailed record of checks, saves, and rests, turn on CSV logging. This will save logs of who performed which check/save or rest and some associated values.


In [17]:
e.set_csv_logging(True)
e.sim_round()
e.sim_quick_rest()

[16:38][DEBUG]: Drew [♦️ 6] vs ♠️ 2 with TR 4: Hit
[16:38][INFO]: 👽 shakes off Burned
[16:38][INFO]: 👽 used 1/2 PP with Attack, Charge
[16:38][DEBUG]: Drew [♠️ 7] vs ♠️ 8 with TR 3: Suited Hit
[16:38][INFO]: 👽 wounded 🎇 by 2: AP 0/2, HP 6/8
[16:38][INFO]: 💀 used 1/1 PP with Attack, Charge
[16:38][DEBUG]: Drew [♠️ J] vs ♠️ 8 with TR 3: Suited Hit
[16:38][INFO]: 💀 wounded 🎇 by 2: AP 0/2, HP 4/8
[16:38][INFO]: 💀 used 0/0 PP with Attack, Dual-Wield
[16:38][DEBUG]: Drew [♦️ 2, ♥️ 9, ♦️ 3] vs ♠️ 8 with TR 3 at Lower Hand 3: Miss
[16:38][INFO]: 🎇 used 0/6 PP with Sear
[16:38][DEBUG]: Drew [♠️ T] vs ♠️ 2 with TR 0: Suited Miss
[16:38][INFO]: 👽 is Burned: 1
[16:38][INFO]: 🎇 used 1/6 PP with Barrier
[16:38][DEBUG]: Drew [♦️ 9] vs ♥️ K with TR 4: Color Hit
[16:38][INFO]: 🎇 wounded 💀 by 1: AP 0/2, HP 7/7
[16:38][DEBUG]: Drew [♦️ 5] vs ♠️ 2 with TR 3: Hit
[16:38][DEBUG]: Recovering 2 with cards
[16:38][DEBUG]:    1 PP to 2
[16:38][DEBUG]:    1 HP to 8
[16:38][INFO]: 👽 recovered 2 HP/PP/AP during Qu

The following uses system commands rather than Python to look at the output. To use this data more meaningfully, try loading the data with [pandas](https://pythonbasics.org/read-csv-with-pandas/).


In [18]:
!head automation/_output/log_draws.csv

date,name,check_save,result_int,result_str,DR,type,mod,upper_lower,draw_n
01/22 09:16:38,👽,check,-3,Miss,2,Brute,1,n,1
01/22 09:16:38,👽,check,-1,Suited Miss,2,Brute,1,n,1
01/22 09:16:38,💀,check,-2,Color Miss,2,Brute,1,n,1
01/22 09:16:38,💀,check,2,Color Hit,2,Brute,1,n,1
01/22 09:16:38,👽,save,-3,Miss,1,INT,-3,n,1
01/22 09:16:38,🎇,check,1,Hit,3,Finesse,1,n,1
01/22 09:16:38,🎇,check,1,Hit,3,Finesse,1,n,1
01/22 09:16:38,👽,check,1,Hit,2,Brute,1,n,1
01/22 09:16:38,👽,save,3,Suited Hit,3,GUT,1,n,1


In [19]:
!head automation/_output/log_rests.csv

date,name,before_after,type,discards,hand,HP,AP,PP,RestCards
01/22 09:16:38,👽,before,quick,15,3,5,0,1,7
01/22 09:16:38,👽,after,quick,16,5,8,0,2,4
01/22 09:16:38,💀,before,quick,16,5,7,1,0,7
01/22 09:16:38,💀,after,quick,18,5,8,1,2,5
01/22 09:16:38,🎇,before,quick,8,1,1,0,5,8
01/22 09:16:38,🎇,after,quick,13,1,8,0,6,3
01/22 09:16:38,👽,before,full,6,5,8,2,2,4
01/22 09:16:38,👽,after,full,1,5,7,2,2,7
01/22 09:16:38,💀,before,full,8,5,8,2,2,5
